# Мини задание 3

Продолжаем работать с задачкой на Kaggle про предсказание клика на рекламу - https://www.kaggle.com/c/avito-context-ad-clicks

В этот раз нужно уже полноценно решить поставленную задачу.

**1. [+5 баллов]** Из всех имеющихся данных необходимо составить датасет из **минимум 8 логических признаков**. Под логическим признаком имеется в виду то, что например кодирование категориального признака или текста в виде OheHot\BoW не считается за кучу признаков - конкретный текст считается за 1 признак, категориальный признак считается за 1 признак.

Всю работу по обработке данных и сборке финального датасета необходимо провести с помощью **спарка**.
Финальный датасет должен быть закодирован в **формате Vowpal Wabbit**.

**Важно** Следите за тем, чтобы исходное количество объектов не изменилось. При неаккуратной работе с join в sql вы можете начать терять какие-то записи, что плохо. Как минимум это означает, что вы не сможете собрать такие же признаки для тестовой выборки, которую предлагают в задаче.

**Важно** Обратите внимание, что в задаче стоит вопрос именно про контекстуальную рекламу, а не про всю существующую.


In [33]:
! sudo -u hdfs hdfs dfsadmin -safemode leave

Safe mode is OFF


In [4]:
import findspark
findspark.init()

In [9]:
import pyspark
from pyspark.sql import SparkSession, Row


conf = pyspark.SparkConf()
conf.set("spark.driver.maxResultSize", "4g")

In [5]:
sc = pyspark.SparkContext(appName="mhw3", conf=conf)
se = SparkSession(sc)

2022-03-14 19:12:50,246 WARN spark.SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4

In [8]:
! hdfs dfs -ls /user/avito/

Found 7 items
-rw-r--r--   1 ubuntu hadoop  5350670676 2022-02-20 22:30 /user/avito/AdsInfo.tsv
-rw-r--r--   1 ubuntu hadoop  9469373867 2022-02-20 22:26 /user/avito/SearchInfo.tsv
-rw-r--r--   1 ubuntu hadoop 13180996392 2022-02-20 22:35 /user/avito/VisitsStream.tsv
drwxr-xr-x   - ubuntu hadoop           0 2022-02-21 12:52 /user/avito/parquet
drwxr-xr-x   - ubuntu hadoop           0 2022-02-21 18:56 /user/avito/results
-rw-r--r--   1 ubuntu hadoop   557829937 2022-02-20 22:26 /user/avito/testSearchStream.tsv
-rw-r--r--   1 ubuntu hadoop  7122681856 2022-02-20 22:29 /user/avito/trainSearchStream.tsv


In [7]:
! hdfs dfs -ls /user/avito/parquet/

Found 4 items
drwxr-xr-x   - ubuntu hadoop          0 2022-02-21 01:00 /user/avito/parquet/AdsInfo.parquet
drwxr-xr-x   - ubuntu hadoop          0 2022-02-21 00:32 /user/avito/parquet/SearchInfo.parquet
drwxr-xr-x   - ubuntu hadoop          0 2022-02-21 12:56 /user/avito/parquet/VisitsStream.parquet
drwxr-xr-x   - ubuntu hadoop          0 2022-02-21 00:42 /user/avito/parquet/trainSearchStream.parquet


In [13]:
! ls  /home/ubuntu/avito-dataset/

AdsInfo.tsv	 PhoneRequestsStream.tsv       testSearchStream.tsv
Category.tsv	 sampleSubmission.csv	       trainSearchStream.tsv
database.sqlite  sampleSubmission_HistCTR.csv  UserInfo.tsv
Location.tsv	 SearchInfo.tsv		       VisitsStream.tsv


In [14]:
! hdfs dfs -put /home/ubuntu/avito-dataset/Category.tsv /user/avito/
! hdfs dfs -put /home/ubuntu/avito-dataset/Location.tsv /user/avito/
! hdfs dfs -put /home/ubuntu/avito-dataset/PhoneRequestsStream.tsv /user/avito/
! hdfs dfs -put /home/ubuntu/avito-dataset/UserInfo.tsv /user/avito/

In [57]:
def write_df(name_df):
    df = se.read.option("mode","DROPMALFORMED") \
                .option('sep', "\t") \
                .csv("/user/avito/{}.tsv".format(name_df), header=True, inferSchema=True)
    df.write.parquet("/user/avito/parquet/{}.parquet".format(name_df))

# write_df('Category')
# write_df('Location')
# write_df('PhoneRequestsStream')
# write_df('UserInfo')

In [58]:
! hdfs dfs -put /home/ubuntu/avito-dataset/testSearchStream.tsv /user/avito/
write_df('testSearchStream')

put: `/user/avito/testSearchStream.tsv': File exists


In [6]:
search_info = se.read.parquet("/user/avito/parquet/SearchInfo.parquet")
search_info.registerTempTable('search_info')

stream_df = se.read.parquet("/user/avito/parquet/trainSearchStream.parquet")
stream_df.registerTempTable('stream_train')

stream_test_df = se.read.parquet("/user/avito/parquet/testSearchStream.parquet")
stream_test_df.registerTempTable('stream_test')

ads_df = se.read.parquet("/user/avito/parquet/AdsInfo.parquet")
ads_df.registerTempTable('ads')

# visits_df = se.read.parquet("/user/avito/parquet/VisitsStream.parquet")
# visits_df.registerTempTable('visits')

category_df = se.read.parquet("/user/avito/parquet/Category.parquet")
category_df.registerTempTable('category')

location_df = se.read.parquet("/user/avito/parquet/Location.parquet")
location_df.registerTempTable('location')

# phone_df = se.read.parquet("/user/avito/parquet/PhoneRequestsStream.parquet")
# phone_df.registerTempTable('phones')

# user_df = se.read.parquet("/user/avito/parquet/UserInfo.parquet")
# user_df.registerTempTable('users')

In [7]:
! sudo -u hdfs hdfs balancer 

2022-03-14 19:13:39,051 INFO balancer.Balancer: namenodes  = [hdfs://rc1a-dataproc-m-5ohwrulcy2ee6jxr.mdb.yandexcloud.net:8020]
2022-03-14 19:13:39,057 INFO balancer.Balancer: parameters = Balancer.BalancerParameters [BalancingPolicy.Node, threshold = 10.0, max idle iteration = 5, #excluded nodes = 0, #included nodes = 0, #source nodes = 0, #blockpools = 0, run during upgrade = false]
2022-03-14 19:13:39,058 INFO balancer.Balancer: included nodes = []
2022-03-14 19:13:39,058 INFO balancer.Balancer: excluded nodes = []
2022-03-14 19:13:39,058 INFO balancer.Balancer: source nodes = []
Time Stamp               Iteration#  Bytes Already Moved  Bytes Left To Move  Bytes Being Moved  NameNode
2022-03-14 19:13:39,063 INFO balancer.NameNodeConnector: getBlocks calls for hdfs://rc1a-dataproc-m-5ohwrulcy2ee6jxr.mdb.yandexcloud.net:8020 will be rate-limited to 20 per second
2022-03-14 19:13:40,366 INFO balancer.Balancer: dfs.namenode.get-blocks.max-qps = 20 (default=20)
2022-03-14 19:13:40,366 IN

In [10]:
se.sql("""
    SELECT *
    FROM stream_train
    WHERE IsClick IS NOT NULL
""").count()

174767397

In [32]:
# HistCTR 
# Position
# ObjectType

# Ads Title
# Ads Price

# SearchQuery

Full Ad: (4)
    AdID, CatLevel, Price, Title

    CatID -> Cat Level

    без params

SearchInfo: (4)
    SearchID, hours, searchQuery, searchCatLevel
    
    searchDate -> hours
    serachDate -> weekday
    searchCat -> searchCatLevel
    SearchQuery
    
full stream:
    HistCTR, Position, ObjectType, Price, Title, hours, searchQuery, searchCatLevel, CatLevel
    

____

# number of phone clicks for an add
# time of the day
# количество просмотров ad

# Ad Location

# level of category
# average price for given category views of a given user

____

# # of ads seen up to a given impression
# # of ads clicked up to a given impression



SyntaxError: invalid syntax (2859797778.py, line 10)

## Оставляем только контекстную рекламу

In [8]:
stream_df = stream_df.filter('IsClick IS NOT NULL').cache()
stream_df.registerTempTable('stream_train')

In [9]:
ads_df = ads_df.filter(ads_df.IsContext == 1).cache()
ads_df.registerTempTable('ads')

In [10]:
se.sql("""
    select *
    from ads
    limit 10
""").toPandas()

,AdID,LocationID,CategoryID,Params,Price,Title,IsContext
0,473,NaN,53,{181:'Для офиса'},69999.0,Кабинет руководителя премиум-класса Han Италия,1
1,1515,NaN,38,{45:'Компьютерные столы и кресла'},8300.0,Стол двухтумбовый Милан-10 - венге-клен,1
2,2247,NaN,4,{156:'Горные'},33900.0,"""1412 FORMAT 26"""" 24 СК""",1
3,3892,NaN,38,{45:'Шкафы и комоды'},12090.0,Детский комплекс Дюймовочка-3.4,1
4,4223,NaN,60,"{178:'Для девочек', 179:'Трикотаж'}",1304.0,Свитер,1
5,5094,NaN,60,"{178:'Для девочек', 179:'Другое'}",539.0,Купальник цельный,1
6,5708,NaN,38,{45:'Столы и стулья'},17980.0,Стол для переговоров прямоугольный Шпон Венге,1
7,7445,NaN,38,{45:'Подставки и тумбы'},1398.0,"""Полка для обуви """"4-Tier Wood Stackable""""""",1
8,8370,NaN,50,{44:'Инструменты'},12313.0,Защитная гильза вибротрамбовки Masalta MR75R,1
9,8790,NaN,38,{45:'Шкафы и комоды'},8900.0,Прихожая Мебелайн-1,1


## AdsInfo

In [13]:
from pyspark.sql import functions as F
import re
import string

def wordify(text):
    if not text:
        return ""
    pattern = re.compile(r"[\w]+")
    result = []

    for match in pattern.finditer(text.lower()):
        word = match.group(0)
        result.append(word)
    return ' '.join(result)

f_wordify = se.udf.register("wordify", wordify, "string")

In [14]:
ads_full = se.sql("""
    SELECT
        l.AdID,
        r.Level as CatLevel,
        l.Price,
        wordify(l.Title) as Title
    FROM ads l
    JOIN category r
    ON l.CategoryID = r.CategoryID
""").cache()
ads_full.registerTempTable('ads_full')

In [15]:
ads_full.toPandas()

,AdID,CatLevel,Price,Title
0,473,3,69999.0,кабинет руководителя премиум класса han италия
1,1515,3,8300.0,стол двухтумбовый милан 10 венге клен
2,2247,3,33900.0,1412 format 26 24 ск
3,3892,3,12090.0,детский комплекс дюймовочка 3 4
4,4223,3,1304.0,свитер
...,...,...,...,...
28531,36890203,3,50000.0,жанровая картина безмятежность
28532,36890207,3,20836.0,кухонный уголок остин т
28533,36890301,3,49990.0,музыкальная система midi panasonic sc max 770gsk
28534,36891249,3,5999.0,стол триал mos f1073 черный


## SearchInfo

In [16]:
from datetime import datetime
def take_hours(text):
    cur_date = datetime.strptime(text, "%Y-%m-%d %H:%M:%S.%f")
    hours = cur_date.strftime('%H')
    return hours

def take_weekday(text):
    cur_date = datetime.strptime(text, "%Y-%m-%d %H:%M:%S.%f")
    weekday = cur_date.weekday()
    return weekday

take_hours = se.udf.register("take_hours", take_hours)
take_weekday = se.udf.register("take_weekday", take_weekday)

In [18]:
search_full = se.sql("""
    SELECT
        l.SearchID,
        take_hours(l.SearchDate) as Hour,
        take_weekday(l.SearchDate) as Weekday,
        wordify(l.SearchQuery) as Query,
        r.Level as SearchCatLevel
    FROM search_info l
    JOIN category r
    ON l.CategoryID = r.CategoryID
""").cache()
search_full.registerTempTable('search_full')

2022-03-14 19:15:23,438 WARN execution.CacheManager: Asked to cache already cached data.


In [19]:
train_df = se.sql("""
    WITH stream_with_ad AS (
        SELECT
            l.SearchID,
            l.Position,
            l.ObjectType,
            l.HistCTR,
            l.IsClick,
            r.CatLevel,
            r.Price,
            r.Title
        FROM stream_train l
        LEFT JOIN ads_full r 
        ON l.AdID = r.AdID
    )
    SELECT
        sl.Position,
        sl.ObjectType,
        sl.HistCTR,
        sl.CatLevel,
        sl.Price,
        sl.Title,
        sr.Hour,
        sr.Weekday,
        sr.Query,
        sr.SearchCatLevel,
        sl.IsClick
    FROM stream_with_ad sl
    LEFT JOIN search_full sr
    ON sl.SearchID = sr.SearchID
""").cache()

In [20]:
train_df.show()

+--------+----------+--------+--------+--------+--------------------+----+-------+-----+--------------+-------+
|Position|ObjectType| HistCTR|CatLevel|   Price|               Title|Hour|Weekday|Query|SearchCatLevel|IsClick|
+--------+----------+--------+--------+--------+--------------------+----+-------+-----+--------------+-------+
|       1|         3|  1.0E-5|       3|   671.0|футболка поло с к...|  17|      1|     |             3|      0|
|       7|         3|  1.0E-5|       3|  2639.0|              куртка|  17|      1|     |             3|      0|
|       1|         3| 0.00634|       3| 12870.0|золотые серьги so...|  18|      6|     |             3|      0|
|       7|         3|0.008191|       3|  6500.0|золотое кольцо но...|  18|      6|     |             3|      0|
|       1|         3|0.012376|       3| 36600.0|детская кровать т...|  20|      2|     |             3|      0|
|       7|         3|0.006342|       3|  1890.0|комплект на выпис...|  20|      2|     |             3| 

In [34]:
train_df.filter(train_df.Query != "").take(5)

[Row(Position=7, ObjectType=3, HistCTR=1e-05, CatLevel=3, Price=3700.0, Title='Беспроводная клавиатура APPLE Wireless Keyboard Bl', Hour='23', Weekday='1', Query='компьютеры', SearchCatLevel=1, IsClick=0),
 Row(Position=1, ObjectType=3, HistCTR=0.029126, CatLevel=3, Price=34990.0, Title='Apple Mac mini (MGEM2RU/A)', Hour='23', Weekday='1', Query='компьютеры', SearchCatLevel=1, IsClick=0),
 Row(Position=1, ObjectType=3, HistCTR=0.005822, CatLevel=3, Price=10010.0, Title='Телевизор LED Mystery MTV-3217LW черный', Hour='20', Weekday='4', Query='led телевизор бу', SearchCatLevel=3, IsClick=0),
 Row(Position=7, ObjectType=3, HistCTR=0.000465, CatLevel=3, Price=44480.0, Title='LED телевизор Philips 49PUS7809/60', Hour='20', Weekday='4', Query='led телевизор бу', SearchCatLevel=3, IsClick=0),
 Row(Position=1, ObjectType=3, HistCTR=0.040747, CatLevel=3, Price=4450.0, Title='Инкубатор Несушка БИ-2', Hour='12', Weekday='4', Query='инкубатор', SearchCatLevel=1, IsClick=0)]

In [21]:
def convert_to_vw(row):
    target = row['IsClick']
    
    title = wordify(row['Title'])
    query = row['Query'] if row['Query'] else "nan"

    template = "{target} \
    |q {query} \
    |t {title} \
    |pos Position:{position} \
    |ot ObjectType:{objectType} \
    |hi HistCTR:{histCTR}\
    |cl CatLevel:{catLevel} \
    |ho Hour:{hour}\
    |w Weekday:{weekday}\
    |sl SearchCatLevel:{searchCatLevel}\
    |pr Price:{price}"
    return template.format(
        target=target,
        query=query,
        title=title,
        position=row['Position'],
        objectType=row['ObjectType'],
        histCTR=row['HistCTR'],
        catLevel=row['CatLevel'],
        hour=row['Hour'],
        weekday=row['Weekday'],
        searchCatLevel=row['SearchCatLevel'],
        price=row['Price']
    )

In [22]:
convert_to_vw(train_df.first())

'0     |q nan     |t футболка поло с короткими рукавами     |pos Position:1     |ot ObjectType:3     |hi HistCTR:1e-05    |cl CatLevel:3     |ho Hour:17    |w Weekday:1    |sl SearchCatLevel:3    |pr Price:671.0'

In [23]:
! hdfs dfs -rm -r /user/avito/datasets/train.vw || true

Deleted /user/avito/datasets/train.vw


In [24]:
(
    train_df
    .rdd
    .map(lambda x: convert_to_vw(x))
    .saveAsTextFile('/user/avito/datasets/train.vw')
)

In [ ]:
! hdfs dfs -cat /user/avito/datasets/train.vw/* >> train_df.vw

## 1. number of phone clicks for an add

In [29]:
ad_phoned = phone_df.select(phone_df.AdID, phone_df.PhoneRequestDate).groupBy(phone_df.AdID).count().cache()

2022-03-13 10:15:44,243 WARN execution.CacheManager: Asked to cache already cached data.


In [31]:
ad_phoned.join(ads_df, ad_phoned.AdID == ads_df.AdID).count()

0

In [17]:
visits_df.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- IPID: integer (nullable = true)
 |-- AdID: integer (nullable = true)
 |-- ViewDate: string (nullable = true)



## 2. number of views for an add

In [22]:
views_df = se.sql("""
    SELECT
    l.AdID AS AdID,
    count(r.ViewDate) AS ViewsCnt
    FROM ads l
    JOIN visits r
    ON l.AdID = r.AdID
    GROUP BY l.AdID
""").cache()
views_df.registerTempTable('views')

In [23]:
views_df.show()

+----+--------+
|AdID|ViewsCnt|
+----+--------+
+----+--------+



In [27]:
ads_df.show()

+-----+----------+----------+--------------------+-------+--------------------+---------+
| AdID|LocationID|CategoryID|              Params|  Price|               Title|IsContext|
+-----+----------+----------+--------------------+-------+--------------------+---------+
|  473|      null|        53|   {181:'Для офиса'}|69999.0|Кабинет руководит...|        1|
| 1515|      null|        38|{45:'Компьютерные...| 8300.0|Стол двухтумбовый...|        1|
| 2247|      null|         4|      {156:'Горные'}|33900.0|"1412 FORMAT 26""...|        1|
| 3892|      null|        38|{45:'Шкафы и комо...|12090.0|Детский комплекс ...|        1|
| 4223|      null|        60|{178:'Для девочек...| 1304.0|              Свитер|        1|
| 5094|      null|        60|{178:'Для девочек...|  539.0|   Купальник цельный|        1|
| 5708|      null|        38|{45:'Столы и стул...|17980.0|Стол для перегово...|        1|
| 7445|      null|        38|{45:'Подставки и ...| 1398.0|"Полка для обуви ...|        1|
| 8370|   

In [28]:
se.sql("""
    SELECT
        *
    FROM ads
    JOIN visits ON ads.AdID = visits.AdID
""").show()

+----+----------+----------+------+-----+-----+---------+------+----+----+--------+
|AdID|LocationID|CategoryID|Params|Price|Title|IsContext|UserID|IPID|AdID|ViewDate|
+----+----------+----------+------+-----+-----+---------+------+----+----+--------+
+----+----------+----------+------+-----+-----+---------+------+----+----+--------+



In [24]:
visits_df.show()

+-------+-------+--------+--------------------+
| UserID|   IPID|    AdID|            ViewDate|
+-------+-------+--------+--------------------+
|3339459|2141522|17013380|2015-05-19 17:47:...|
|3472056| 468478| 9250676|2015-05-19 17:47:...|
|3498500|2194994|16265489|2015-05-19 17:47:...|
|3581440| 167293|29431869|2015-05-19 17:47:...|
|3756375|1294847|  311552|2015-05-19 17:47:...|
|3891751|1190064| 9709139|2015-05-19 17:47:...|
|3971739|1811955|16457203|2015-05-19 17:47:...|
|4006974|1796040|36742059|2015-05-19 17:47:...|
|4102599| 678975|32139132|2015-05-19 17:47:...|
|4106979|1479340|32656590|2015-05-19 17:47:...|
|4203357|1759496| 2785886|2015-05-19 17:47:...|
|4271481| 604680| 2311288|2015-05-19 17:47:...|
|  29832|1519127| 2774307|2015-05-19 17:47:...|
| 106382| 698275|24729353|2015-05-19 17:47:...|
| 232031| 351316| 3087932|2015-05-19 17:47:...|
| 263551|1706098| 3345028|2015-05-19 17:47:...|
| 296159|1597739| 9021206|2015-05-19 17:47:...|
| 323952| 290722|  974185|2015-05-19 17:

In [14]:
# average price for given category views of a given user
# avg_price = search_info.join(ads_df, search_info.CategoryID == ads_df.CategoryID).select(search_info.UserID, ads_df.CategoryID, ads_df.Price)
# avg_price = avg_price.groupBy('UserID', 'CategoryID').agg({"Price": "avg"}).cache()
# avg_price.take(10)

In [12]:
# # time of the day
# # day of the week

# from datetime import datetime
# def take_hours(text):
#     cur_date = datetime.strptime(text, "%Y-%m-%d %H:%M:%S.%f")
#     hours = cur_date.strftime('%H')
#     return hours

# def take_weekday(text):
#     cur_date = datetime.strptime(text, "%Y-%m-%d %H:%M:%S.%f")
#     weekday = cur_date.weekday()
#     return weekday

# take_hours = se.udf.register("take_hours", take_hours)
# take_weekday = se.udf.register("take_weekday", take_weekday)

# visits_days = se.sql("""
#     SELECT DISTINCT
#         UserID,
#         take_hours(ViewDate) as Hour,
#     FROM visits
# """).cache()
# visits_days.registerTempTable("days")
# visits_days.show()

In [12]:
search_info.printSchema()
stream_df.printSchema()

root
 |-- SearchID: integer (nullable = true)
 |-- SearchDate: string (nullable = true)
 |-- IPID: integer (nullable = true)
 |-- UserID: integer (nullable = true)
 |-- IsUserLoggedOn: integer (nullable = true)
 |-- SearchQuery: string (nullable = true)
 |-- LocationID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- SearchParams: string (nullable = true)

root
 |-- SearchID: integer (nullable = true)
 |-- AdID: integer (nullable = true)
 |-- Position: integer (nullable = true)
 |-- ObjectType: integer (nullable = true)
 |-- HistCTR: double (nullable = true)
 |-- IsClick: integer (nullable = true)



In [11]:
stream_df = stream_df.filter("IsClick IS NOT NULL").cache()

In [46]:
stream_with_query = stream_df.join(search_info, search_info.SearchID == stream_df.SearchID, 'left')
stream_with_query = stream_with_query.select('SearchQuery', 'AdID', 'Position', 'ObjectType', 'HistCTR', 'IsClick').cache()


In [47]:
stream_with_query.printSchema()

root
 |-- SearchQuery: string (nullable = true)
 |-- AdID: integer (nullable = true)
 |-- Position: integer (nullable = true)
 |-- ObjectType: integer (nullable = true)
 |-- HistCTR: double (nullable = true)
 |-- IsClick: integer (nullable = true)



In [48]:
ads_df.printSchema()

root
 |-- AdID: integer (nullable = true)
 |-- LocationID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- Params: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- IsContext: integer (nullable = true)



In [49]:
ads_with_phone = ads_df.join(ad_phoned, ads_df.AdID == ad_phoned.AdID, 'left').select(ads_df.AdID, 'Price', 'Title', 'IsContext', 'count').cache()

In [50]:
ads_with_phone.printSchema()

root
 |-- AdID: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- IsContext: integer (nullable = true)
 |-- count: long (nullable = true)



In [51]:
train_df = stream_with_query.join(ads_with_phone, ads_with_phone.AdID == stream_with_query.AdID, 'left')
train_df = train_df.select('SearchQuery', 'Position', 'ObjectType', 'HistCTR', 'Price', 'Title', 'IsContext', 'count', 'IsClick').cache()

In [52]:
train_df.printSchema()

root
 |-- SearchQuery: string (nullable = true)
 |-- Position: integer (nullable = true)
 |-- ObjectType: integer (nullable = true)
 |-- HistCTR: double (nullable = true)
 |-- Price: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- IsContext: integer (nullable = true)
 |-- count: long (nullable = true)
 |-- IsClick: integer (nullable = true)



In [54]:
train_df = train_df.na.fill({'count':0, 'SearchQuery': ""}).cache()

In [55]:
train_df.show()

+-----------+--------+----------+-------+-----+-----+---------+-----+-------+
|SearchQuery|Position|ObjectType|HistCTR|Price|Title|IsContext|count|IsClick|
+-----------+--------+----------+-------+-----+-----+---------+-----+-------+
|           |       7|         3| 1.0E-5|458.0|2топа|        1|    0|      0|
|           |       7|         3| 1.0E-5|458.0|2топа|        1|    0|      0|
|           |       1|         3| 1.0E-5|458.0|2топа|        1|    0|      0|
|           |       7|         3| 1.0E-5|458.0|2топа|        1|    0|      0|
|           |       7|         3| 1.0E-5|458.0|2топа|        1|    0|      0|
|           |       7|         3| 1.0E-5|458.0|2топа|        1|    0|      0|
|           |       7|         3| 1.0E-5|458.0|2топа|        1|    0|      0|
|   комплект|       7|         3| 1.0E-5|458.0|2топа|        1|    0|      0|
|   комплект|       7|         3| 1.0E-5|458.0|2топа|        1|    0|      0|
|           |       1|         3| 1.0E-5|458.0|2топа|        1| 

In [60]:
stream_test_with_query = stream_test_df.join(search_info, search_info.SearchID == stream_test_df.SearchID, 'left')
stream_test_with_query = stream_test_with_query.select('SearchQuery', 'AdID', 'Position', 'ObjectType', 'HistCTR').cache()

test_df = stream_test_with_query.join(ads_with_phone, ads_with_phone.AdID == stream_test_with_query.AdID, 'left')
test_df = test_df.select('SearchQuery', 'Position', 'ObjectType', 'HistCTR', 'Price', 'Title', 'IsContext', 'count')
test_df = test_df.na.fill({'count':0, 'SearchQuery': ""}).cache()

In [61]:
test_df.show()

+--------------------+--------+----------+-------+---------+--------------------+---------+-----+
|         SearchQuery|Position|ObjectType|HistCTR|    Price|               Title|IsContext|count|
+--------------------+--------+----------+-------+---------+--------------------+---------+-----+
|                    |       8|         1|   null|1600000.0|Камаз 65117 зерновоз|        0|    1|
|              нексия|       2|         1|   null|   1050.0|Фонарь задний Дэу...|        0|    0|
|              нексия|       8|         1|   null|   1050.0|Фонарь задний Дэу...|        0|    0|
|                    |       6|         1|   null|   8000.0|Продам резину на УАЗ|        0|    0|
|                    |       8|         1|   null|   8000.0|Продам резину на УАЗ|        0|    0|
|                 уаз|       8|         1|   null|   8000.0|Продам резину на УАЗ|        0|    0|
|                    |       6|         1|   null| 200000.0|  Ваш надежный мопед|        0|    0|
|                   

In [42]:
from pandarallel import pandarallel
pandarallel.initialize(use_memory_fs=False, progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [91]:
avg_price_train = train_df.agg({'Price': 'avg'}).collect()[0]['avg(Price)']
avg_price_test = test_df.agg({'Price': 'avg'}).collect()[0]['avg(Price)']

In [92]:
train_df = train_df.na.fill({'HistCTR':0, 'Title': "", 'Price': avg_price_train}).cache()
test_df = test_df.na.fill({'HistCTR':0, 'Title': "", 'Price': avg_price_test}).cache()

In [100]:
train_df.write.parquet("/user/avito/parquet/train_df_2.parquet")
test_df.write.parquet("/user/avito/parquet/test_df_2.parquet")

In [3]:
train_df = se.read.parquet("/user/avito/parquet/train_df_2.parquet").cache()
test_df = se.read.parquet("/user/avito/parquet/test_df_2.parquet").cache()

In [62]:
! sudo wget http://finance.yendor.com/ML/VW/Binaries/vw-8.20190624 -O /usr/bin/vw
! sudo chmod +x /usr/bin/vw
! sudo chown ubuntu /usr/bin/vw

--2022-03-12 16:09:17--  http://finance.yendor.com/ML/VW/Binaries/vw-8.20190624
Resolving finance.yendor.com (finance.yendor.com)... 69.163.152.190
Connecting to finance.yendor.com (finance.yendor.com)|69.163.152.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9467376 (9.0M)
Saving to: ‘/usr/bin/vw’

/usr/bin/vw         100%[===================>]   9.03M  4.15MB/s    in 2.2s    

2022-03-12 16:09:20 (4.15 MB/s) - ‘/usr/bin/vw’ saved [9467376/9467376]



In [63]:
! sudo apt-get update -y && sudo apt-get install graphviz -y
! pip install -q numpy pandas sklearn dateparser pandarallel ipywidgets catboost graphviz

Hit:1 http://storage.yandexcloud.net/dataproc2x/ci/2-0-alpha-12 focal InRelease
Hit:2 https://repo.saltproject.io/py3/ubuntu/20.04/amd64/3002 focal InRelease  
Hit:3 http://mirror.yandex.ru/ubuntu focal InRelease                           
Get:4 http://mirror.yandex.ru/ubuntu focal-updates InRelease [114 kB]          
Get:5 https://repos.influxdata.com/ubuntu focal InRelease [4,736 B]            
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:7 http://mirror.yandex.ru/ubuntu focal-backports InRelease [108 kB]        
Get:8 http://mirror.yandex.ru/mirrors/postgresql focal-pgdg InRelease [86.6 kB]
Get:9 http://mirror.yandex.ru/ubuntu focal-updates/main amd64 Packages [1,641 kB]
Get:10 https://repos.influxdata.com/ubuntu focal/stable amd64 Packages [1,267 B]
Get:11 http://mirror.yandex.ru/ubuntu focal-updates/main i386 Packages [616 kB]
Get:12 http://mirror.yandex.ru/ubuntu focal-updates/main Translation-en [312 kB]
Get:13 http://mirror.yandex.ru/ubunt

Setting up libgvpr2:amd64 (2.42.2-3build2) ...
Setting up libxmu6:amd64 (2:1.1.3-0ubuntu1) ...
Setting up libpango-1.0-0:amd64 (1.44.7-2ubuntu4) ...
Setting up libxaw7:amd64 (2:1.0.13-1) ...
Setting up libpangoft2-1.0-0:amd64 (1.44.7-2ubuntu4) ...
Setting up libpangocairo-1.0-0:amd64 (1.44.7-2ubuntu4) ...
Setting up libgvc6 (2.42.2-3build2) ...
Setting up graphviz (2.42.2-3build2) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for libc-bin (2.31-0ubuntu9.2) ...
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if 

In [64]:
! /opt/conda/bin/jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [65]:
! vw --help | head

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1
driver:
  --onethread           Disable parse thread
VW options:
  --ring_size arg (=256, ) size of example ring
  --strict_parse           throw on malformed examples
Update options:
  -l [ --learning_rate ] arg Set learning rate
  --power_t arg              t power value
  --decay_learning_rate arg  Set Decay factor for learning_rate between passes
  --initial_t arg            initial t value


In [5]:
import pandas as pd
import numpy as np
import re
import dateparser
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
train_df.count()

174767397

In [9]:
test_df.count()

15961515

In [11]:
convert_to_vw(train_df.first())

'0 |sq      |t  |pos position:7     |ot objectType:3 |h histCTR:0.000485    |pr price:1099.0 |ic isContext:1     |cnt count:0'

In [115]:
se.udf.register("convert_to_vw", convert_to_vw)

<function __main__.convert_to_vw(row)>

In [30]:
def write_vw(rdd_, filename):
    with open(filename, 'w') as f:
        for row_ in rdd_:
            f.write(convert_to_vw(row_) + '\n')

In [4]:
train_df.toPandas()

ERROR:root:Exception while sending command.                        (5 + 5) / 10]
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:41019)
Traceback (most recent call last):
  File "/usr/lib/spark/python/pyspark/sql/dataframe.py", line 596, in colle

#
# java.lang.OutOfMemoryError: Java heap space
# -XX:OnOutOfMemoryError="kill -9 %p"
#   Executing /bin/sh -c "kill -9 7826"...


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:41019)

In [ ]:
train_rdd = train_df.rdd.collect()
test_rdd = test_df.rdd.collect()

In [29]:
train_rdd.take(2)

[Row(SearchQuery='отдам', Position=7, ObjectType=3, HistCTR=0.000485, Price=1099.0, Title='Платье и леггинсы', IsContext=1, count=0, IsClick=0),
 Row(SearchQuery='отдам', Position=1, ObjectType=3, HistCTR=1e-05, Price=1099.0, Title='Платье и леггинсы', IsContext=1, count=0, IsClick=0)]

In [15]:
write_vw(train_df, 'avito.train.vw')

In [22]:
! hdfs dfs -ls /user/avito/datasets

-rw-r--r--   1 ubuntu hadoop 5350670676 2022-02-20 22:30 /user/avito/AdsInfo.tsv


In [ ]:
! hdfs dfs -rm -r /user/tweets/spark/top10 || true

In [16]:
write_vw(test_df, 'avito.test.vw')

2022-03-12 18:55:04,553 WARN scheduler.TaskSetManager: Lost task 4.0 in stage 11.0 (TID 39, rc1a-dataproc-c-dh5md77ip9o1c50g.mdb.yandexcloud.net, executor 1): org.apache.spark.SparkException: Task failed while writing rows
	at org.apache.spark.internal.io.SparkHadoopWriter$.executeTask(SparkHadoopWriter.scala:162)
	at org.apache.spark.internal.io.SparkHadoopWriter$.$anonfun$write$1(SparkHadoopWriter.scala:88)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.run(Task.scala:127)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$3(Executor.scala:462)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:465)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache

Py4JJavaError: An error occurred while calling o98.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:105)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1090)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:388)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1088)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1061)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope

In [ ]:
train_df.show()


**2. [+5 баллов]** Разделите ваш датасет на обучающую и тестовую выборку. Обучите **логистическую регрессию** с помощью **Vowpal Wabbit**. 

Сделайте бинарные предсказания и посчитайте Accuracy, Presicion и Recall на тестовой выборке. 
Сделайте предсказания вероятностей и посчитайте LogLoss. (Эта же метрика используется в Kaggle для оценивания вашего решения).

Посчитайте ответы для тестовой выборки, которая предлагается в задании и отправьте в Kaggle (само соревнование уже давно закончилось, но вы все еще можете присылать туда свои решения и система будет вас оценивать). В решении приложите скриншот того, какой скор вам выдал Kaggle.



In [99]:
df_train, df_test = train_test_split(train_df, test_size=0.2, random_state=420)

TypeError: Expected sequence or array-like, got <class 'pyspark.sql.dataframe.DataFrame'>

**3. [+ up to 5 дополнительных баллов]** Несмотря на то, что мы просим вас считать качество (и на тестовой и в Kaggle), оно никак **не будет влиять** на основные баллы. Основная цель этого задания состоит в том, чтобы вы потренировались в End-to-end работе с ML задачей на больших данных - от сбора датасета до получения модели и ее оценки. 

Однако, топ 5 студентов (в каждой группе отдельно) с самым больших качеством (по Kaggle) получат до 5 дополнительных баллов в зависимости от полученой метрики качества (за самое лучшее качество +5, за второе место +4 и так далее).

**Важно** В сдаваемом ноубуке должны присутствовать такие элементы
* Код на Spark, который собирает датасет
* Команды запуска VW для его обучения
* Подсчитанные метрики качества (и соответственно код, который их считает)
* Скриншот из Kaggle, на котором видно вашу посылку и подсчитанную метрику качества